In [1]:
f = open('US_births_1994-2003_CDC_NCHS.csv', 'r')
data = f.read().split('\n')
print(data[:10])

['year,month,date_of_month,day_of_week,births', '1994,1,1,6,8096', '1994,1,2,7,7772', '1994,1,3,1,10142', '1994,1,4,2,11248', '1994,1,5,3,11053', '1994,1,6,4,11406', '1994,1,7,5,11251', '1994,1,8,6,8653', '1994,1,9,7,7910']


In [2]:
def read_csv(csv):
    f = open(csv, 'r')
    data = f.read().split('\n')
    string_list = data[1:]
    final_list = []
    for row in string_list:
        int_fields = []
        string_fields = row.split(',')
        for row in string_fields:
            int_fields.append(int(row))
        final_list.append(int_fields)
    return final_list

In [3]:
cdc_list = read_csv('US_births_1994-2003_CDC_NCHS.csv')
cdc_list[:10]

[[1994, 1, 1, 6, 8096],
 [1994, 1, 2, 7, 7772],
 [1994, 1, 3, 1, 10142],
 [1994, 1, 4, 2, 11248],
 [1994, 1, 5, 3, 11053],
 [1994, 1, 6, 4, 11406],
 [1994, 1, 7, 5, 11251],
 [1994, 1, 8, 6, 8653],
 [1994, 1, 9, 7, 7910],
 [1994, 1, 10, 1, 10498]]

In [4]:
def month_births(lol):
    births_per_month = {}
    for row in lol:
        month = row[1]
        births = row[4]
        if month in births_per_month:
            births_per_month[month] += births
        else:
            births_per_month[month] = births
    return births_per_month

In [5]:
cdc_month_births = month_births(cdc_list)
cdc_month_births

{1: 3232517,
 2: 3018140,
 3: 3322069,
 4: 3185314,
 5: 3350907,
 6: 3296530,
 7: 3498783,
 8: 3525858,
 9: 3439698,
 10: 3378814,
 11: 3171647,
 12: 3301860}

In [6]:
def dow_births(lol):
    births_per_dow = {}
    for row in lol:
        dow = row[3]
        births = row[4]
        if dow in births_per_dow:
            births_per_dow[dow] += births
        else:
            births_per_dow[dow] = births
    return births_per_dow

In [7]:
cdc_day_births = dow_births(cdc_list)
cdc_day_births

{1: 5789166,
 2: 6446196,
 3: 6322855,
 4: 6288429,
 5: 6233657,
 6: 4562111,
 7: 4079723}

In [8]:
def calc_counts(data, column):
    births_per_col = {}
    for row in data:
        col = row[column]
        births = row[4]
        if col in births_per_col:
            births_per_col[col] += births
        else:
            births_per_col[col] = births
    return births_per_col

In [9]:
cdc_year_births = calc_counts(cdc_list, 0)
cdc_month_births = calc_counts(cdc_list, 1)
cdc_dom_births = calc_counts(cdc_list, 2)
cdc_dow_births = calc_counts(cdc_list, 3)
cdc_dow_births

{1: 5789166,
 2: 6446196,
 3: 6322855,
 4: 6288429,
 5: 6233657,
 6: 4562111,
 7: 4079723}

In [10]:
def min_max(dict):
    min = dict.copy().popitem()[1]
    max = 0
    for key, val in dict.items():
        if val < min:
            min = val
        if val > max:
            max = val
    output = {'max' : max, 'min' : min}
    return output

In [11]:
print(min_max(cdc_dow_births))

{'min': 4079723, 'max': 6446196}


In [12]:
def consec_changes(data, column, col_value):
    start_year = data[0][0]
    change_year = {}
    val_year = {}
    for row in data:
        year = row[0]
        births = row[4]
        if row[column] == col_value:
            if year in val_year:
                val_year[year] += births
            else:
                val_year[year] = births
    for key, value in val_year.items():
        if key != start_year:
            change_year[key] = value - val_year[key - 1]
    return val_year
consec_changes(cdc_list, 3, 7)

{1994: 428752,
 1995: 425790,
 1996: 413336,
 1997: 404478,
 1998: 407129,
 1999: 401991,
 2000: 416454,
 2001: 397119,
 2002: 391375,
 2003: 393299}

In [13]:
ssa_list = read_csv('US_births_2000-2014_SSA.csv')
comb_ssa_cdc_list = read_csv('US_births_1994-2003_CDC_NCHS.csv')
merge_start = 0
count = -1
for row in comb_ssa_cdc_list:
    if row[0] == ssa_list[merge_start][0]:
        row[4] = int(round((row[4] + ssa_list[merge_start][4])/2, 0))
        merge_start += 1
    count += 1
for row in ssa_list:
    if row[0] > comb_ssa_cdc_list[count][0]:
        comb_ssa_cdc_list.append(row)